In [1]:
import ee
service_account = 'renosterveld-ee@ee-vegetation-gee4geo.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-vegetation-gee4geo-6309a79ef209.json')
ee.Initialize(credentials,project='ee-vegetation-gee4geo')
    
import sys
import os
import json

In [2]:
from utils.globals import *
from utils.s2_ee import *
from utils.misc_ee import *
from utils.harmonic_ee  import *


## Training and Validation data (2016-2019)

In [3]:
#observational data
renoster = ee.FeatureCollection(TRAIN_INPUT)\
  .map(num2date)

#turn this into  a raster mask
mask_renoster = renoster \
  .map(lambda feature: feature.set('flag', ee.Number(1))) \
  .reduceToImage(['flag'],ee.Reducer.first())

#map to dates and create imagecol from results
ilist = dates.map(getS2(renoster,mask_renoster))
imageCol_gaps=ee.ImageCollection(ilist)\
  .select(bands+['label'])

#description of timeseries
datelist= dates.map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()

#dates for output
dt_datelist = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in datelist]
datelist_out = []
for d in dt_datelist:
  datelist_out.append(str(d.date()))

tempLength = len(datelist_out)


In [47]:
#It may be necessary to save intermediate fiels to prevent exceeding ee capacity

assetPath =  'users/glennwithtwons/s2_tf'
#str_name = 'task_phase1_'
#saveAssets(imageCol_gaps,assetPath,tempLength,str_name,datelist_out)
imageCol_gaps = ee.ImageCollection(assetPath)\
    .map(dateset).sort('system:time_start')

In [24]:
#form standarizeation of tf input 
icol_min= imageCol_gaps.min()
icol_max= imageCol_gaps.max()
Max = eeminmax(95,icol_max,1)
Min = eeminmax(5,icol_min,0)

with open('max.json', 'w') as fp:
    json.dump(Max, fp)

with open('min.json', 'w') as fp:
    json.dump(Min, fp)

In [48]:
names = imageCol_gaps.first().bandNames()
#fill non null value in series
first_im = imageCol_gaps\
  .reduce(ee.Reducer.firstNonNull())\
  .rename(names)\
  .set({'system:index': 'first'})

first = ee.List([first_im])

#fill gaps then drop first image
imageCol = ee.ImageCollection(ee.List(imageCol_gaps.iterate(gap_fill, first)))\
  .filter(ee.Filter.neq('system:index', 'first'))

imageCol = imageCol.map(remclip(mask_renoster))
tsLength= imageCol.size().getInfo()
#149
#flatten each band
imageCol_all = imageCol.toArrayPerBand()

In [56]:
#It may be necessary to save intermediate fiels to prevent exceeding ee capacity

#assetPath = 'users/glennwithtwons/s2_reno_CNN_dltraining_6_flat_2020-08-04'
assetPath = 'users/glennwithtwons/s2_reno_CNN_dltraining_6_flat_2021-02-08'
#str_name = 'task_phase2_'
#task = ee.batch.Export.image.toAsset(image=imageCol_all,
#                                 region=aoi,
#                                 description=str_name,
#                                 assetId=assetPath,
#                                 scale=10,
#                                 maxPixels=2000000000000)
#task.start()
#task.status()
imageCol_all = ee.Image(assetPath)

In [67]:
#Create random points. num points = num pixels x fact
#geo_aoi = renoster.geometry()
#num = ee.Number(geo_aoi.area()).divide(100).multiply(0.25).round()
#rpoints = ee.FeatureCollection.randomPoints(geo_aoi,num,42)

#OR

#load precreated points for every pixel
rpoints = ee.FeatureCollection(EXTRACT_POINTS)

#add random number to feature to select polygons at random
random = renoster.randomColumn('random',42)
random_poly = random.reduceToImage(['random'],ee.Reducer.first())
random_poly = random_poly.select(['first'],['random_geom'])

#add flag for points as real or not
rfilter = ee.Image(FILTER_TAB)

#a random image to select pixels at random
random_pix = ee.Image.random(44)

# Sample the image
imageCol_all = imageCol_all.addBands(random_poly).addBands(rfilter).addBands(random_pix)
sample = rpoints.map(lambda feature: feature.set(\
                         imageCol_all.reduceRegion(reducer='first',\
                                                   geometry=feature.geometry(),\
                                                   scale=10)))
# random subsample at sites of change
real = sample.filter(ee.Filter.eq('sum', ee.Number(2)))\
.filter(ee.Filter.lt('random', SPLIT2))
# random subsample at sites without change
notreal = sample.filter(ee.Filter.eq('sum', ee.Number(1)))\
.filter(ee.Filter.lt('random', SPLIT3))

sample_thin = real.merge(notreal)

# Partition the sample based on polygons
training = sample_thin.filter(ee.Filter.lt('random_geom', SPLIT))
valid = sample_thin.filter(ee.Filter.gte('random_geom', SPLIT))

# Create the task to export tfrecord data
trainingTask = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=trainFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord',
  selectors=featureNames)

validTask = ee.batch.Export.table.toCloudStorage(
  collection=valid,
  description='Valid Export',
  fileNamePrefix=validFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord',
  selectors=featureNames)

# Start the tasks.
trainingTask.start()
#validTask.start()
trainingTask.status()
#validTask.status()

## Test data (2020)

In [3]:
renoster = ee.FeatureCollection(TEST_INPUT)\
  .map(num2date)

#turn this into  a raster mask
mask_renoster = renoster \
  .map(lambda feature: feature.set('flag', ee.Number(1))) \
  .reduceToImage(['flag'],ee.Reducer.first())

#map to dates and create imagecol from results
ilist = t_dates.map(getS2(renoster,mask_renoster))
imageCol_gaps=ee.ImageCollection(ilist)\
  .select(bands+['label'])

#description of timeseries
datelist= t_dates.map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()

#dates for output
dt_datelist = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in datelist]
datelist_out = []
for d in dt_datelist:
  datelist_out.append(str(d.date()))

tempLength = len(datelist_out)

#harmonics
imageCol_gaps = imageCol_gaps\
.map(addDependents)\
.map(addHarmonics(harmonicFrequencies))

#one for each index
imageCol_gaps = harmon_resid('ndvi',imageCol_gaps)\
.select(allbands+hbands+['label'])

#count excceednaces
#first image
count = ee.Image(0).rename('count').toInt()
roll = ee.Image(0).rename('roll').toInt()
first_ex =  imageCol_gaps\
  .reduce(ee.Reducer.first())\
  .addBands(count)\
  .addBands(roll)\
  .set({'system:index': 'first_ex'})
first_ex = ee.List([first_ex])

#iterate
imageCol_gaps = ee.ImageCollection(ee.List(imageCol_gaps.iterate(roll_count, first_ex)))\
  .filter(ee.Filter.neq('system:index', 'first_ex'))

#fill non null value in series
names = imageCol_gaps.first().bandNames()
first_im = imageCol_gaps\
  .reduce(ee.Reducer.firstNonNull())\
  .rename(names)\
  .set({'system:index': 'first'})

first = ee.List([first_im])

#fill gaps then drop first image
imageCol = ee.ImageCollection(ee.List(imageCol_gaps.iterate(gap_fill, first)))\
  .filter(ee.Filter.neq('system:index', 'first'))

#flatten
imageCol = imageCol\
.map(remclip(mask_renoster))\
.filterDate(TDATESTART,TDATEEND)

tesLength = imageCol.size().getInfo()
#flatten each band
imageCol_all = imageCol.toArrayPerBand()

In [4]:
#It may be necessary to save intermediate fiels to prevent exceeding ee capacity

#assetPath_test = 'users/glennwithtwons/s2_reno_CNN_test_export2'
#task = ee.batch.Export.image.toAsset(image=imageCol_all,
#                                    region=aoi,
#                                    description="testing",
#                                    assetId=assetPath_test,
#                                    scale=10,
#                                    maxPixels=2000000000000)
#task.start()
#task.status()
#imageCol_all = ee.Image(assetPath_test)

In [ ]:
#create random points
#geo_aoi = renoster.geometry()
#num = ee.Number(geo_aoi.area()).divide(100).multiply(0.25).round()
#rpoints = ee.FeatureCollection.randomPoints(geo_aoi,num,42)

#OR

#load precreated points for every pixel
rpoints = ee.FeatureCollection(TEST_POINTS)\
.randomColumn('random',42)\
.filter(ee.Filter.lt('random', TEST_SPLIT));

# Sample the image
testing = rpoints.map(lambda feature: feature.set(\
                         imageCol_all.reduceRegion(reducer='first',\
                                                   geometry=feature.geometry(),\
                                                   scale=10)))
testNames = imageCol_all.bandNames().getInfo()

# Create the tasks.
testTask = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Test Export',
  fileNamePrefix=testFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord',
  selectors=testNames)

testTask.start()
#testTask.status()